In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
from folium import Map, CircleMarker, Vega, Popup,IFrame
import base64
from rpy2.robjects import r, numpy2ri
numpy2ri.activate()
from rpy2.robjects.packages import importr
sinkr = importr('sinkr')

In [2]:
data=pd.read_csv('v100_hr_20200101-0930_cut_utf8.csv')

In [3]:
###
#X
###
u=np.zeros([244,8520])

for i in range (0,1704):
    u[0][i]=np.nan

for i in range (0,243):
    a=np.array(data['cal_PMf'][i*6816:(i*6816)+24])
    for j in range (1,71):
        b=np.array(data['cal_PMf'][(i*6816)+(96*j):(i*6816)+(96*j)+24])
        a=np.append(a, b)
    for k in range(1,1704):
        if a[k]=='\\N' :
            a[k]=np.nan
    for l in range (0,1704):
        u[i+1][l]=a[l]

for i in range (0,244):
    a=np.array(data['cal_PMf'][6816*i:6816*i+6816])
    for j in range(0,6816):
        if a[j]=='\\N' :
            a[j]=np.nan
    for l in range (1704,8520):
        u[i][l]=a[l-1704]
###
#Y
###
v=np.zeros([244,6816])
for i in range (0,244):
    a=np.array(data['obs_PMf'][6816*i:6816*i+6816])
    for j in range(0,6816):
        if a[j]=='\\N' :
            a[j]=np.nan
    v[i]=a.T
###
#station
###
station=np.zeros([71,2])
for i in range (0,71):
    station[i][0]=np.array(data['lat'][96*i])
    station[i][1]=np.array(data['lon'][96*i])
# u=u.T
# print(station)

#print(np.shape(station))
print(np.shape(u))
print(np.shape(v))

(244, 8520)
(244, 6816)


# DINEOF (like FPCA)

In [4]:
XRestruct=sinkr.dineof(u)
YRestruct=sinkr.dineof(v)

[1]
 "1 EOF ; RMS = 10.81159056"


[1]
 "1 EOF ; RMS = 10.79075511"


[1]
 "1 EOF ; RMS = 10.79041738"


[1]
 "1 EOF ; RMS = 10.79048848"


[1]
 "2 EOF ; RMS = 9.9731839"


[1]
 "2 EOF ; RMS = 9.9726094"


[1]
 "2 EOF ; RMS = 9.97285047"


[1]
 "3 EOF ; RMS = 9.69047753"


[1]
 "3 EOF ; RMS = 9.69482565"


[1]
 "4 EOF ; RMS = 9.43015115"


[1]
 "4 EOF ; RMS = 9.43247906"


[1]
 "5 EOF ; RMS = 9.19173886"


[1]
 "5 EOF ; RMS = 9.1948205"


[1]
 "6 EOF ; RMS = 9.04020923"


[1]
 "6 EOF ; RMS = 9.0483692"


[1]
 "7 EOF ; RMS = 8.93501465"


[1]
 "7 EOF ; RMS = 8.93945984"


[1]
 "8 EOF ; RMS = 8.88704742"


[1]
 "8 EOF ; RMS = 8.90642001"


[1]
 "9 EOF ; RMS = 8.84125912"


[1]
 "9 EOF ; RMS = 8.85342476"


[1]
 "10 EOF ; RMS = 8.79394935"


[1]
 "10 EOF ; RMS = 8.79918436"


[1]
 "11 EOF ; RMS = 8.78018407"


[1]
 "11 EOF ; RMS = 8.81056698"


[1]
 "12 EOF ; RMS = 8.79448762"


[1]
 "12 EOF ; RMS = 8.82397063"


[1]
 "13 EOF ; RMS = 8.7833032"


[1]
 "13 EOF ; RMS = 8.80037508"


[1]
 "1

# Restruct Function

In [5]:
XRestruct_Fun=np.array(XRestruct[0])
YRestruct_Fun=np.array(YRestruct[0])
print(np.shape(XRestruct_Fun))
print(np.shape(YRestruct_Fun))

(244, 8520)
(244, 6816)


In [6]:
XCov_fun=np.cov(XRestruct_Fun.T)
print(np.shape(XCov_fun))

(8520, 8520)


In [7]:
XMean_fun=np.zeros(8520)
for i in range(0,8520):
    for j in range(0,244):
        XMean_fun[i]=XMean_fun[i]+(XRestruct_Fun[j][i]/244)
YMean_fun=np.zeros(6816)
for i in range(0,6816):
    for j in range(0,244):
        YMean_fun[i]=YMean_fun[i]+(YRestruct_Fun[j][i]/244)
print(np.shape(XMean_fun))
print(np.shape(YMean_fun))

(8520,)
(6816,)


In [8]:
Xhat=XRestruct_Fun
Yhat=YRestruct_Fun
Xbar=XMean_fun
Ybar=YMean_fun
print('Xhat')
#print(Xhat)
print(np.shape(Xhat))
print('Yhat')
#print(Yhat)
print(np.shape(Yhat))
print('Xbar')
#print(Xbar)
print(np.shape(Xbar))
print('Ybar')
#print(Ybar)
print(np.shape(Ybar))

Xhat
(244, 8520)
Yhat
(244, 6816)
Xbar
(8520,)
Ybar
(6816,)


# Regression

In [9]:
nXCov_fun=np.zeros([6816,6816])
for i in range (1704,8520):
    for j in range (1704,8520):
        nXCov_fun[i-1704][j-1704]=XCov_fun[i][j]

G_xx=nXCov_fun
G_xxt=np.zeros(6816)
for i in range(0,6816):
     G_xxt[i]=G_xx[i][i]
print(G_xxt)
print(np.shape(G_xxt))

G_xy=0
# for i in range (0,244):
#     G_xyi=(Xhat[i]-Xbar)*(Yhat[i]-Ybar)
#     G_xy=G_xy+G_xyi
# G_xy=G_xy/243
G_xy=np.cov(Xhat.T,Yhat.T)
print(G_xy)
print(np.shape(G_xy))

[ 68.15802914  63.24933945 116.31025274 ...  98.99547747 118.30559943
 139.82751368]
(6816,)
[[ 88.49661921  74.03814033  81.13381645 ...  -2.90060036  -3.64480207
   -2.98852343]
 [ 74.03814033  85.24888308  96.4424401  ...  -5.79922629  -4.07075548
   -4.08978987]
 [ 81.13381645  96.4424401  150.39634966 ...  -9.3419944   -4.9008665
   -6.056324  ]
 ...
 [ -2.90060036  -5.79922629  -9.3419944  ... 190.37186203 181.70854018
  190.28440021]
 [ -3.64480207  -4.07075548  -4.9008665  ... 181.70854018 197.15749331
  203.67973024]
 [ -2.98852343  -4.08978987  -6.056324   ... 190.28440021 203.67973024
  233.08514237]]
(15336, 15336)


In [10]:
# B_1=G_xy/G_xxt
# B_0=YMean_fun-(B_1*XMean_fun)
# B_1const=0
# for i in range (0,6816):
#     B_1const=B_1const+B_1[i]
# B_1const=B_1const/6816

x1=np.linalg.inv(np.dot(Xhat.T,Xhat))
x2=np.dot(Xhat.T,Yhat)
B_1=np.dot(x1,x2)
print('x1')
print(x1)
print(np.shape(x1))
# for i in range(0,244):
#     for j in range(0,6816):
#         B_1[i][j+1704]=x2[i][j]

B_0=YMean_fun-np.dot(XMean_fun,B_1)

print('B_1')
#print(B_1)
print(np.shape(B_1))
print('B_0')
#print(B_0)
print(np.shape(B_0))

x1
[[-8.04938537e+08  2.30304070e+09 -1.36010824e+08 ... -2.53791461e+08
  -1.37476239e+08 -1.44715860e+09]
 [ 5.14315514e+08 -7.64142791e+08  2.82088751e+08 ...  2.20263319e+08
   1.03561230e+08  7.71238482e+08]
 [ 3.82829348e+07 -9.37939518e+08 -1.24313408e+07 ...  3.99412794e+08
   2.98217997e+08 -6.01633064e+05]
 ...
 [-1.26808660e+08 -1.17891632e+08 -2.79312933e+08 ... -1.56776517e+07
   1.35401974e+08 -1.87442085e+08]
 [ 3.02948609e+08 -1.38607910e+09 -2.04299285e+08 ...  3.45594626e+08
  -5.34664378e+07  6.09530377e+08]
 [ 3.76264466e+08 -3.89179184e+08  3.11657682e+08 ...  6.53768678e+07
   8.32837448e+07  3.34381332e+08]]
(8520, 8520)
B_1
(8520, 6816)
B_0
(6816,)
